In [1]:
%load_ext kedro.ipython

[06/24/24 22:52:32] INFO     Registered line magic '%reload_kedro'                                   ]8;id=325986;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=980345;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#58\58]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=553914;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=941602;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#60\60]8;;\

                    INFO     Resolved project path as: /Users/wilsvenleong/Downloads/Learning       ]8;id=949790;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=471442;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#171\171]8;;\
                             Materials/healthhub-content-optimization/content-optimization.                        
                             To set a different path, run '%reload_kedro <project_root>'                           

[06/24/24 22:52:34] INFO     Kedro project Content Optimization                                     ]8;id=771122;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=988777;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#141\141]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=35462;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=76634;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#142\142]8;;\
                             'pipelines'                                                                           

[06/24/24 22:52:35] INFO     Registered line magic 'run_viz'                                        ]8;id=176878;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=545447;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#148\148]8;;\

In [2]:
import re
import unicodedata

from bs4 import BeautifulSoup

In [3]:
def clean_text(text: str) -> str:
    # Normalize Unicode characters
    text = unicodedata.normalize("NFKD", text)

    # Replace common problematic characters
    text = text.replace("\xa0", " ")  # non-breaking space
    text = text.replace("\u200b", "")  # zero-width space
    text = text.replace("\u2028", "\n")  # line separator
    text = text.replace("\u2029", "\n")  # paragraph separator
    # Replace multiple whitespace with single space
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def extract_content(html_content: str) -> str:
    soup = BeautifulSoup(html_content, "html.parser")

    # Find all <br> tags and replace them with newline
    for br in soup.find_all("br"):
        br.replace_with("\n")

    related_sections = []
    read_these_next_ul = None
    # Extract "Related:" sections and "Read these next:" items
    for tag in soup.find_all(["p", "ul"]):
        if tag.name == "p" and tag.find("strong"):
            if "Related:" in tag.text:
                related_sections.append(re.sub(r"Related: ", "", clean_text(tag.text)))
            elif "Read these next:" in tag.text:
                read_these_next_ul = tag.find_next_sibling("ul")
        elif tag == read_these_next_ul:
            for li in tag.find_all("li"):
                related_sections.append(clean_text(li.text))

    # Extract the main content
    content = []
    for tag in soup.find_all(["h2", "h3", "h4", "p", "ul", "ol"]):
        if tag.name in ["h2", "h3", "h4"]:
            content.append(clean_text(tag.text))

        elif tag.name == "p":
            # Remove all em tags
            for em in tag.find_all("em"):
                em.extract()

            # Get the remaining text
            text = tag.get_text()

            # Remove sentences about HealthHub app, Google Play, and Apple Store
            if not re.search(
                r"(HealthHub app|Google Play|Apple Store|Parent Hub)", text
            ):
                if tag.find("strong"):
                    if "Related:" in tag.text:
                        text = clean_text(tag.text)
                        content.append(re.sub(r"\n", " ", text))

                    elif "Read these next:" in tag.text:
                        content.append(clean_text(tag.text))
                else:
                    content.append(clean_text(text))

        # For unordered lists
        elif (
            tag.name == "ul" and tag.parent.name == "div"
        ):  # not "ul" so we avoid duplicates
            for li in tag.find_all("li"):
                content.append("- " + clean_text(li.text))

        # For ordred lists
        elif tag.name == "ol":
            for i, li in enumerate(tag.find_all("li")):
                content.append(f"{i + 1}. " + clean_text(li.text))

        content.append("")  # Add a blank line after each element

    # Remove empty strings from content
    content = [c for c in content if c]

    return related_sections, "\n".join(content)

In [4]:
# ruff: noqa: F821
catalog.list()


[
    'all_contents',
    'all_contents_extracted',
    'all_extracted_text',
    'parameters',
    'params:content_category',
    'params:columns_to_keep',
    'params:columns_to_keep.cost-and-financing',
    'params:columns_to_keep.diseases-and-conditions',
    'params:columns_to_keep.health-statistics',
    'params:columns_to_keep.live-healthy-articles',
    'params:columns_to_keep.medical-care-and-facilities',
    'params:columns_to_keep.medications',
    'params:columns_to_keep.program-sub-pages',
    'params:columns_to_keep.programs',
    'params:columns_to_keep.support-group-and-others',
    'params:metadata',
    'params:metadata.cost-and-financing',
    'params:metadata.cost-and-financing.content_title',
    'params:metadata.cost-and-financing.content_body',
    'params:metadata.diseases-and-conditions',
    'params:metadata.diseases-and-conditions.content_title',
    'params:metadata.diseases-and-conditions.content_body',
    'params:metadata.health-statistics',
    'params:

### Load all the contents

In [5]:
# ruff: noqa: F821
all_contents = catalog.load("all_contents")
all_contents

[06/24/24 22:52:52] INFO     Loading data from all_contents (PartitionedDataset)...             ]8;id=736184;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=685538;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py#508\508]8;;\


{
    'export-published-cost-and-financing_14062024_data.xlsx': <bound method AbstractVersionedDataset.load of <kedro_datasets.pandas.excel_dataset.ExcelDataset object at 0x1679cffb0>>,
    'export-published-diseases-and-conditions_13062024_data.xlsx': <bound method AbstractVersionedDataset.load of <kedro_datasets.pandas.excel_dataset.ExcelDataset object at 0x1679cf3e0>>,
    'export-published-health-statistics_14062024_data.xlsx': <bound method AbstractVersionedDataset.load of <kedro_datasets.pandas.excel_dataset.ExcelDataset object at 0x1679ce9f0>>,
    'export-published-live-healthy-articles_14062024_data.xlsx': <bound method AbstractVersionedDataset.load of <kedro_datasets.pandas.excel_dataset.ExcelDataset object at 0x167a48c20>>,
    'export-published-medical-care-and-facilities_14062024_data.xlsx': <bound method AbstractVersionedDataset.load of <kedro_datasets.pandas.excel_dataset.ExcelDataset object at 0x1679ce300>>,
    'export-published-medications_14062024_data.xlsx': <bound

### Load the content category

In [6]:
# ruff: noqa: F821
content_category = catalog.load("params:content_category")
content_category

[06/24/24 22:52:54] INFO     Loading data from params:content_category (MemoryDataset)...       ]8;id=50120;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=864963;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py#508\508]8;;\

'live-healthy-articles'

### Use `content_category` as key to access the corresponding content

In [7]:
df = all_contents[f"export-published-{content_category}_14062024_data.xlsx"]()
# Drop all columns which have only null values
df = df.dropna(axis=1, how="all")
print(df.shape)

(1155, 60)


### View all columns to keep for corresponding content category

In [8]:
# ruff: noqa: F821
columns_to_keep = catalog.load("params:columns_to_keep")
columns_to_keep

[06/24/24 22:53:33] INFO     Loading data from params:columns_to_keep (MemoryDataset)...        ]8;id=977866;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=610793;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py#508\508]8;;\


{
    'cost-and-financing': [
        'id',
        'Content.Name',
        'CostAndFinancing_Title',
        'CostAndFinancing_ArticleCatNames',
        'CostAndFinancing_CoverImgUrl',
        'CostAndFinancing_FullUrl',
        'CostAndFinancing_FullUrl2',
        'CostAndFinancing_FriendlyUrl',
        'CostAndFinancing_CategoryDesc',
        'CostAndFinancing_ContentBody',
        'CostAndFinancing_ENKeywords',
        'CostAndFinancing_FeatureTitle',
        'CostAndFinancing_PRName',
        'CostAndFinancing_AlternateImageText',
        'CostAndFinancing_DateModified',
        'CostAndFinancing_NumberofViews',
        'CostAndFinancing_LastMonthViewCount',
        'CostAndFinancing_LastTwoMonthsView',
        'Page Views',
        'Engagement Rate',
        'Bounce Rate',
        'Exit Rate',
        'Scroll %',
        '% of Total Views',
        'Cumulative % of Total Views'
    ],
    'diseases-and-conditions': [
        'id',
        'Content.Name',
        'DiseasesConditi

### Get columns to drop corresponding to `content_category`

In [9]:
relevant_columns = columns_to_keep[content_category]
relevant_columns


[
    'id',
    'Content.Name',
    'LiveHealthyArticle_Title',
    'LiveHealthyArticle_ArticleCateName',
    'LiveHealthyArticle_CoverImgUrl',
    'LiveHealthyArticle_FullUrl',
    'LiveHealthyArticle_FullUrl2',
    'LiveHealthyArticle_FriendlyUrl',
    'LiveHealthyArticle_CategoryDes',
    'LiveHealthyArticle_Content_Body',
    'LiveHealthyArticle_ENKeywords',
    'LiveHealthyArticle_FeatureTitle',
    'LiveHealthyArticle_PRName',
    'LiveHealthyArticle_AlternateImageText',
    'LiveHealthyArticle_DateModified',
    'LiveHealthyArticle_Number_of_View',
    'LiveHealthyArticle_Lastmonthview',
    'LiveHealthyArticle_LastTwoMonthsView',
    'Page Views',
    'Engagement Rate',
    'Bounce Rate',
    'Exit Rate',
    'Scroll %',
    '% of Total Views',
    'Cumulative % of Total Views'
]

In [10]:
# Keep all relevant columns
df = df[relevant_columns]
print(df.shape)

(1155, 25)


In [11]:
# Remaining columns
df.columns


Index(['id', 'Content.Name', 'LiveHealthyArticle_Title',
       'LiveHealthyArticle_ArticleCateName', 'LiveHealthyArticle_CoverImgUrl',
       'LiveHealthyArticle_FullUrl', 'LiveHealthyArticle_FullUrl2',
       'LiveHealthyArticle_FriendlyUrl', 'LiveHealthyArticle_CategoryDes',
       'LiveHealthyArticle_Content_Body', 'LiveHealthyArticle_ENKeywords',
       'LiveHealthyArticle_FeatureTitle', 'LiveHealthyArticle_PRName',
       'LiveHealthyArticle_AlternateImageText',
       'LiveHealthyArticle_DateModified', 'LiveHealthyArticle_Number_of_View',
       'LiveHealthyArticle_Lastmonthview',
       'LiveHealthyArticle_LastTwoMonthsView', 'Page Views', 'Engagement Rate',
       'Bounce Rate', 'Exit Rate', 'Scroll %', '% of Total Views',
       'Cumulative % of Total Views'],
      dtype='object')

### Drop all articles with no content

In [12]:
df = df[
    df["LiveHealthyArticle_Content_Body"].apply(
        lambda x: True if re.search(r"(<[div|p|h2].*?>)", x) else False
    )
].reset_index(drop=True)
print(df.shape)

(1149, 25)


In [13]:
tmp = df.sample(5, random_state=42).reset_index(drop=True)
tmp

,id,Content.Name,LiveHealthyArticle_Title,LiveHealthyArticle_ArticleCateName,LiveHealthyArticle_CoverImgUrl,LiveHealthyArticle_FullUrl,LiveHealthyArticle_FullUrl2,LiveHealthyArticle_FriendlyUrl,LiveHealthyArticle_CategoryDes,LiveHealthyArticle_Content_Body,...,LiveHealthyArticle_Number_of_View,LiveHealthyArticle_Lastmonthview,LiveHealthyArticle_LastTwoMonthsView,Page Views,Engagement Rate,Bounce Rate,Exit Rate,Scroll %,% of Total Views,Cumulative % of Total Views
0,1443022,How To Identify And Deal With Depression,How To Identify And Deal With Depression,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/how-to-i...,www.healthhub.sg/live-healthy/how-to-identify-...,how-to-identify-and-deal-with-depression,"Learn how to spot the signs of depression, get...","<div class=""ExternalClass46FF4530AAF64858A79C3...",...,27637.0,1100.0,1947.0,1622,0.863296,0.136704,0.549711,0.364365,0.001371,0.575815
1,1444928,Signs of Stress: Could Stress Be Good for You?,Signs of Stress: Could Stress Be Good for You?,"Mind and Balance,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/stress_c...,www.healthhub.sg/live-healthy/stress_can_be_go...,stress_can_be_good_for_you,"Stressful situations, whether at school or at ...","<div class=""ExternalClassA58275AB1DD8443085927...",...,1468.0,129.0,159.0,331,0.883784,0.116216,0.000000,0.306647,0.000280,0.938865
2,1442901,Coping with your two-year-old: when a firm han...,Coping with your two-year-old: when a firm han...,NaN,https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/baby-cop...,www.healthhub.sg/live-healthy/baby-coping-with...,baby-coping-with-your-2-year-old-when-a-firm-h...,Your two-year-old can be a handful. But here a...,"<div class=""ExternalClass5E6AB2968188425D9CEDB...",...,295.0,42.0,81.0,650,0.808219,0.191781,0.583815,0.366923,0.000549,0.825824
3,1445194,Common Eye Conditions in Your Pre-Schooler,Common Eye Conditions in Your Pre-Schooler,"Body Care,Child and Teen Health,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/eye-care...,www.healthhub.sg/live-healthy/eye-care-for-pre...,eye-care-for-pre-schooler,Learn about the common eye conditions your chi...,"<div class=""ExternalClass4D426A5EDDCF40BA8C223...",...,0.0,0.0,0.0,1244,0.831234,0.168766,0.531587,0.398513,0.001051,0.655277
4,1445895,Managing Wandering Dementia Patients,Managing Wandering Dementia Patients,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/managing...,www.healthhub.sg/live-healthy/managing-wanderi...,managing-wandering-behaviour-in-dementia,One of your family members has just been diagn...,"<div class=""ExternalClassB401DB0539BD4790885E5...",...,712.0,21.0,32.0,413,0.891648,0.108352,0.199074,0.336562,0.000349,0.912451


### Get metadata for corresponding content category

In [14]:
# ruff: noqa: F821
metadata = catalog.load("params:metadata")
metadata

[06/24/24 22:54:38] INFO     Loading data from params:metadata (MemoryDataset)...               ]8;id=945015;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=610965;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py#508\508]8;;\


{
    'cost-and-financing': {
        'content_title': 'CostAndFinancing_Title',
        'content_body': 'CostAndFinancing_ContentBody'
    },
    'diseases-and-conditions': {
        'content_title': 'DiseasesCondition_Title',
        'content_body': 'DiseasesCondition_ContentBody'
    },
    'health-statistics': {
        'content_title': 'HealthStatistics_Title',
        'content_body': 'HealthStatistics_ContentBody'
    },
    'live-healthy-articles': {
        'content_title': 'LiveHealthyArticle_Title',
        'content_body': 'LiveHealthyArticle_Content_Body'
    },
    'medical-care-and-facilities': {
        'content_title': 'MedicalCareFaci_Title',
        'content_body': 'MedicalCareFaci_ContentBody'
    },
    'medications': {'content_title': 'Medication_Title', 'content_body': 'Medication_ContentBody'},
    'program-sub-pages': {
        'content_title': 'ProgramSubPages_Title',
        'content_body': 'ProgramSubPages_Content_Body'
    },
    'programs': {'content_title'

In [15]:
content_title = metadata[content_category]["content_title"]
content_body = metadata[content_category]["content_body"]

print(content_title)
print(content_body)

LiveHealthyArticle_Title
LiveHealthyArticle_Content_Body


In [16]:
tmp["related_sections"] = None
tmp["extracted_content_body"] = None

for index, row in tmp.iterrows():
    title = row[content_title]
    print(title)

    html_content = row[content_body]
    related_sections, extracted_content_body = extract_content(html_content)

    tmp.at[index, "related_sections"] = related_sections
    tmp.at[index, "extracted_content_body"] = extracted_content_body

How To Identify And Deal With Depression
Signs of Stress: Could Stress Be Good for You?
Coping with your two-year-old: when a firm hand is needed
Common Eye Conditions in Your Pre-Schooler
Managing Wandering Dementia Patients


In [17]:
tmp.iloc[0]["related_sections"]


[
    'Myths and Misconceptions About Depression',
    'Coping with Depression',
    'Be a Master of Stress',
    'Feeling Depressed? You Are Not Alone',
    'Helping Youth Deal With Depression',
    'Myths and Misconceptions About Depression'
]

In [18]:
tmp.iloc[0]["extracted_content_body"]

'Signs of Depression\nIt is normal to feel sad when we lose a loved one, fail an exam, or even end a relationship. However, when feelings of sadness is prolonged and affects our daily lives, it is symptomatic of depression.\nOther signs of depression include:\n- A loss of interest in activities previously enjoyed\n- Weight loss or weight gain; or decrease or increase in appetite\n- Difficulty falling asleep or staying asleep; or sleeping excessively\n- Feeling agitated or restless\n- Feeling tired and lacking the energy\n- Feelings of worthlessness or excessive guilt\n- Difficulty concentrating or having trouble thinking\n- Frequent thoughts of death or suicide\nRelated: Myths and Misconceptions About Depression\nGetting Help\nIf you have been constantly feeling sad and experiencing some or all of the symptoms above, it is appropriate to seek professional medical help, especially so if thoughts of death or suicide are present. Do not self-diagnose or self-manage without speaking to a m

In [19]:
tmp

,id,Content.Name,LiveHealthyArticle_Title,LiveHealthyArticle_ArticleCateName,LiveHealthyArticle_CoverImgUrl,LiveHealthyArticle_FullUrl,LiveHealthyArticle_FullUrl2,LiveHealthyArticle_FriendlyUrl,LiveHealthyArticle_CategoryDes,LiveHealthyArticle_Content_Body,...,LiveHealthyArticle_LastTwoMonthsView,Page Views,Engagement Rate,Bounce Rate,Exit Rate,Scroll %,% of Total Views,Cumulative % of Total Views,related_sections,extracted_content_body
0,1443022,How To Identify And Deal With Depression,How To Identify And Deal With Depression,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/how-to-i...,www.healthhub.sg/live-healthy/how-to-identify-...,how-to-identify-and-deal-with-depression,"Learn how to spot the signs of depression, get...","<div class=""ExternalClass46FF4530AAF64858A79C3...",...,1947.0,1622,0.863296,0.136704,0.549711,0.364365,0.001371,0.575815,"[Myths and Misconceptions About Depression, Co...",Signs of Depression\nIt is normal to feel sad ...
1,1444928,Signs of Stress: Could Stress Be Good for You?,Signs of Stress: Could Stress Be Good for You?,"Mind and Balance,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/stress_c...,www.healthhub.sg/live-healthy/stress_can_be_go...,stress_can_be_good_for_you,"Stressful situations, whether at school or at ...","<div class=""ExternalClassA58275AB1DD8443085927...",...,159.0,331,0.883784,0.116216,0.000000,0.306647,0.000280,0.938865,"[_x000D_ Exam Stress Busting Tips, _x000D_ 6 M...",Stress Can Be Your Friend\nWhen examinations a...
2,1442901,Coping with your two-year-old: when a firm han...,Coping with your two-year-old: when a firm han...,NaN,https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/baby-cop...,www.healthhub.sg/live-healthy/baby-coping-with...,baby-coping-with-your-2-year-old-when-a-firm-h...,Your two-year-old can be a handful. But here a...,"<div class=""ExternalClass5E6AB2968188425D9CEDB...",...,81.0,650,0.808219,0.191781,0.583815,0.366923,0.000549,0.825824,[],How to Deal with Two Year Old Tantrums?\nAs ba...
3,1445194,Common Eye Conditions in Your Pre-Schooler,Common Eye Conditions in Your Pre-Schooler,"Body Care,Child and Teen Health,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/eye-care...,www.healthhub.sg/live-healthy/eye-care-for-pre...,eye-care-for-pre-schooler,Learn about the common eye conditions your chi...,"<div class=""ExternalClass4D426A5EDDCF40BA8C223...",...,0.0,1244,0.831234,0.168766,0.531587,0.398513,0.001051,0.655277,"[Healthy Eyes, Clear Vision, Different Spectac...","In Singapore, children are also becoming myopi..."
4,1445895,Managing Wandering Dementia Patients,Managing Wandering Dementia Patients,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/managing...,www.healthhub.sg/live-healthy/managing-wanderi...,managing-wandering-behaviour-in-dementia,One of your family members has just been diagn...,"<div class=""ExternalClassB401DB0539BD4790885E5...",...,32.0,413,0.891648,0.108352,0.199074,0.336562,0.000349,0.912451,[My Loved One Has Dementia. What Do I Do?],Wandering is common among people with dementia...
